In [ ]:
pip install opencv-python pillow numpy tk

Note: you may need to restart the kernel to use updated packages.


**Yolo v3 CFG file:**
https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg

**Yolo v3 Names File**
https://github.com/pjreddie/darknet/blob/master/data/coco.names

**Yolo v3 Weights**
https://github.com/patrick013/Object-Detection---Yolov3/blob/master/model/yolov3.weights

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk
from PIL import Image, ImageTk
import cv2
import numpy as np
import threading

class YOLOFaceDetectionApp:
# Initialize the main application window and set up the layout and widgets
    def __init__(self, root):
        self.root = root
        self.root.title("YOLO Object Detection and Filters")
        self.root.geometry("800x600")
        self.root.configure(bg="#2C3E50")

        self.panel = tk.Label(root, bg="#34495E")
        self.panel.pack(padx=10, pady=10)

        btn_frame = tk.Frame(root, bg="#2C3E50")
        btn_frame.pack(fill=tk.X, pady=10)
# Create and add buttons to the button frame
        self.create_button(btn_frame, "Select Weights", self.select_weights, "#3498DB")
        self.create_button(btn_frame, "Select CFG", self.select_cfg, "#1ABC9C")
        self.create_button(btn_frame, "Select Names", self.select_names, "#E74C3C")
        self.create_button(btn_frame, "Select Image", self.select_image, "#F1C40F")
        self.create_button(btn_frame, "Select Video", self.select_video, "#9B59B6")
        self.create_button(btn_frame, "Live Video", self.toggle_live_video, "#E67E22")
        self.create_button(btn_frame, "Detect Objects", self.toggle_detect_objects, "#16A085")
        self.create_button(btn_frame, "Clear", self.clear_filters, "#95A5A6")

        filter_frame = tk.Frame(root, bg="#2C3E50")
        filter_frame.pack(fill=tk.X, pady=10)
 # Create and add filter selection dropdown
        filter_label = tk.Label(filter_frame, text="Filter", bg="#2C3E50", fg="white", font=("Arial", 12, "bold"))
        filter_label.pack(side=tk.LEFT, padx=10)
        self.filter_var = tk.StringVar()
        self.filter_menu = ttk.Combobox(filter_frame, textvariable=self.filter_var, state="readonly")
        self.filter_menu['values'] = ("Edge Detection", "Sharpen", "Low Pass", "Gaussian", "High Pass", "Emboss",
                                      "Median", "Gaussian Blur", "Box Blur", "Identity", "Sepia", "Negative", "Cartoon")
        self.filter_menu.pack(side=tk.LEFT, padx=10)
        self.filter_menu.bind("<<ComboboxSelected>>", self.apply_filter)

        self.weights_path = None
        self.cfg_path = None
        self.names_path = None
        self.image_path = None
        self.video_path = None
        self.image = None
        self.video_capture = None
        self.net = None
        self.classes = None
        self.output_layers = None
        self.filter_mode = None
        self.detect_objects_flag = False
        self.running = False
        self.thread = None
# Helper function to create a button with specified text, command, and background color
    def create_button(self, frame, text, command, bg_color):
        btn = tk.Button(frame, text=text, command=command, bg=bg_color, fg="white", font=("Arial", 12, "bold"), relief="raised")
        btn.pack(side=tk.LEFT, padx=10, pady=5)
# Open file dialog to select YOLO weights file and load YOLO model if all files are selected
    def select_weights(self):
        self.weights_path = filedialog.askopenfilename(filetypes=[("Weights files", "*.weights")])
        self.load_yolo()
# Open file dialog to select YOLO config file and load YOLO model if all files are selected
    def select_cfg(self):
        self.cfg_path = filedialog.askopenfilename(filetypes=[("Config files", "*.cfg")])
        self.load_yolo()
 # Open file dialog to select YOLO class names file and load YOLO model if all files are selected
    def select_names(self):
        self.names_path = filedialog.askopenfilename(filetypes=[("Names files", "*.names")])
        self.load_yolo()
 # Load YOLO model if weights, config, and names files are selected
    def load_yolo(self):
        if self.weights_path and self.cfg_path and self.names_path:
            try:
                self.net = cv2.dnn.readNet(self.weights_path, self.cfg_path)
                self.layer_names = self.net.getLayerNames()
                self.output_layers = [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()]
                with open(self.names_path, "r") as f:
                    self.classes = [line.strip() for line in f.readlines()]
                messagebox.showinfo("YOLO", "YOLO model loaded successfully.")
            except Exception as e:
                messagebox.showerror("YOLO Error", f"Error loading YOLO: {e}")
 # Open file dialog to select an image and load it
    def select_image(self):
        self.image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.jpeg")])
        if self.image_path:
            self.load_image()
# Open file dialog to select a video and start object detection in a new thread
    def select_video(self):
        self.video_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4;*.avi;*.mov")])
        if self.video_path:
            if self.running:
                self.stop_running()
            else:
                self.running = True
                self.thread = threading.Thread(target=self.detect_objects_video)
                self.thread.start()
 # Start or stop live video feed for object detection
    def toggle_live_video(self):
        if self.running:
            self.stop_running()
        else:
            self.video_capture = cv2.VideoCapture(0)
            self.running = True
            self.thread = threading.Thread(target=self.show_live_video)
            self.thread.start()
# Load the selected image, apply filter, detect objects, and display it
    def load_image(self):
        image = cv2.imread(self.image_path)
        if image is None:
            messagebox.showerror("Image Error", "Failed to load image.")
            return

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.apply_filter_to_frame(image)

        if self.detect_objects_flag:
            image = self.apply_yolo(image)

        image = Image.fromarray(image)
        image = ImageTk.PhotoImage(image)

        self.panel.config(image=image)
        self.panel.image = image

    def apply_filter(self, event):
        filter_name = self.filter_var.get()
        self.filter_mode = filter_name
        if self.image_path:
            self.load_image()

    def clear_filters(self):
        self.filter_mode = "Identity"
        self.filter_var.set("Identity")
        if self.image_path:
            self.load_image()

    def toggle_detect_objects(self):
        self.detect_objects_flag = not self.detect_objects_flag
        if self.image_path:
            self.load_image()

    def apply_yolo(self, frame):
        if self.net is None:
            raise ValueError("YOLO model is not loaded.")

        height, width, _ = frame.shape
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        self.net.setInput(blob)
        outs = self.net.forward(self.output_layers)

        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                for obj in detection:
                    scores = obj[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5:
                        center_x = int(obj[0] * width)
                        center_y = int(obj[1] * height)
                        w = int(obj[2] * width)
                        h = int(obj[3] * height)
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)

                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(self.classes[class_ids[i]])
                confidence = confidences[i]
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        return frame

    def apply_filter_to_frame(self, frame):
        if self.filter_mode == "Edge Detection":
            kernel = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])
            return cv2.filter2D(frame, -1, kernel)
        elif self.filter_mode == "Sharpen":
            kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
            return cv2.filter2D(frame, -1, kernel)
        elif self.filter_mode == "Low Pass":
            return cv2.blur(frame, (5, 5))
        elif self.filter_mode == "Gaussian":
            return cv2.GaussianBlur(frame, (15, 15), 0)
        elif self.filter_mode == "High Pass":
            kernel = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])
            return cv2.filter2D(frame, -1, kernel)
        elif self.filter_mode == "Emboss":
            kernel = np.array([[0, -1, -1], [1, 0, -1], [1, 1, 0]])
            return cv2.filter2D(frame, -1, kernel)
        elif self.filter_mode == "Median":
            return cv2.medianBlur(frame, 5)
        elif self.filter_mode == "Gaussian Blur":
            return cv2.GaussianBlur(frame, (5, 5), 0)
        elif self.filter_mode == "Box Blur":
            return cv2.blur(frame, (5, 5))
        elif self.filter_mode == "Identity":
            return frame
        elif self.filter_mode == "Sepia":
            sepia_filter = np.array([[0.272, 0.534, 0.131],
                                     [0.349, 0.686, 0.168],
                                     [0.393, 0.769, 0.189]])
            return cv2.transform(frame, sepia_filter)
        elif self.filter_mode == "Negative":
            return cv2.bitwise_not(frame)
        elif self.filter_mode == "Cartoon":
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray = cv2.medianBlur(gray, 5)
            edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)
            color = cv2.bilateralFilter(frame, 9, 250, 250)
            return cv2.bitwise_and(color, color, mask=edges)
        return frame
# Continuously capture frames from live video feed, apply filters and object detection, and display them
    def show_live_video(self):
        while self.running:
            ret, frame = self.video_capture.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = self.apply_filter_to_frame(frame)
                if self.detect_objects_flag:
                    frame = self.apply_yolo(frame)
                frame = Image.fromarray(frame)
                frame = ImageTk.PhotoImage(frame)

                self.panel.config(image=frame)
                self.panel.image = frame
# Continuously capture frames from the selected video, apply filters and object detection, and display them
    def detect_objects_video(self):
        cap = cv2.VideoCapture(self.video_path)
        while self.running:
            ret, frame = cap.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = self.apply_filter_to_frame(frame)
                if self.detect_objects_flag:
                    frame = self.apply_yolo(frame)
                frame = Image.fromarray(frame)
                frame = ImageTk.PhotoImage(frame)

                self.panel.config(image=frame)
                self.panel.image = frame

    def stop_running(self):
        self.running = False
        if self.thread is not None:
            self.thread.join()
        if self.video_capture is not None:
            self.video_capture.release()

if __name__ == "__main__":
    root = tk.Tk()
    app = YOLOFaceDetectionApp(root)
    root.mainloop()
